# Profiling dask vs numpy #

[Dask](http://dask.pydata.org/en/latest/index.html) is a powerful new tool for parallel processing in python. This notebooks does some benchmarking comparisons with regular numpy and suggests how astropy could be modified to take advantage of dask multithreading. (It should release the GIL more.) These tests were conducted on a server with 32 cores. 

In [ ]:
import numpy as np
import dask.array as da

## Create a large random numpy array ##

In [ ]:
Nz,Ny,Nx = (64,1024,2048)
shape = (Nz,Ny,Nx)
big_numpy_array = np.random.rand(*shape)

## Reduce operation using numpy ##

In [ ]:
%time (big_numpy_array**2).sum()

## Create a dask array from the numpy array ##
cs is the chunksize, so we use 64 different chunks of about 1 million elements

In [ ]:
cs = (1,Ny,Nx)
big_dask_array = da.from_array(big_numpy_array, chunks=cs)

In [ ]:
%time (big_dask_array**2).sum().compute()

We can see that we got some multithreaded acceleration, because the Wall time is less than the CPU time. This speedup is possible because the numpy ufuncs [release the global interpreter lock (GIL)](http://docs.scipy.org/doc/numpy-dev/reference/internals.code-explanations.html) at the cython level.

In [ ]:
# do the same thing using map_blocks
def map_func(b):
    return (b**2)

%time big_dask_array.map_blocks(map_func).sum().compute()

## FFT example ##
Here we also get speedup with numpy.fft because it evidently also releases the GIL.

In [ ]:
# try with fft
% time np.fft.fft2(big_numpy_array).sum()

In [ ]:
% time big_dask_array.map_blocks(np.fft.fft2).sum().compute()
# huge speedup!

## Gaussian Filter - scipy.ndimage ##
This does get multethreaded speeup.

In [ ]:
from scipy.ndimage import gaussian_filter

def filter_func(b):
    return gaussian_filter(b.squeeze(), 1)[np.newaxis,:,:]

# this does speed up because it releases the GIL
% time big_dask_array.map_blocks(filter_func).sum().compute()

## Gaussian Filter - astropy - upstream/master ##
Astropy convolution is understandably slower because it checks for missing data. But on top of this, it does NOT get multethreaded speeup because the [astropy convolution functions](https://github.com/astropy/astropy/blob/master/astropy/convolution/boundary_fill.pyx) do NOT release the GIL before their loops. As a result, it is nearly 60 times slow than ndimage when used with dask.

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.6-x86_64-3.5/')

from astropy.convolution import convolve, Gaussian2DKernel
import astropy
astropy.version

In [ ]:
def filter_func_ap(b):
    ker = Gaussian2DKernel(1)
    return convolve(b.squeeze(), ker, boundary='extend')[np.newaxis,:,:]

In [ ]:
%time big_dask_array.map_blocks(filter_func_ap).sum().compute()

In [ ]:
# test single threaded execution
%timeit filter_func_ap(big_numpy_array[0])

## switch to my fork ##

In [ ]:
%time big_dask_array.map_blocks(filter_func_ap).sum().compute()

In [ ]:
%timeit filter_func_ap(big_numpy_array[0])

In [ ]:
import numpy as np
import dask.array as da
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.6-x86_64-3.5/')

from astropy.convolution import convolve, Gaussian2DKernel
import astropy
print(astropy.version)
sys.stdout.flush()

Nz,Ny,Nx = (64,1024,2048)
shape = (Nz,Ny,Nx)
big_numpy_array = np.random.rand(*shape)

cs = (1,Ny,Nx)
big_dask_array = da.from_array(big_numpy_array, chunks=cs)

def filter_func_ap(b):
    ker = Gaussian2DKernel(1)#, x_size=111, y_size=111)
    return convolve(b.squeeze(), ker, boundary=None)[np.newaxis,:,:]


In [ ]:
%time big_dask_array.map_blocks(filter_func_ap).sum().compute()

In [ ]:
%timeit filter_func_ap(big_numpy_array[0])

In [ ]:
810/760

In [ ]:
216/206


In [1]:
import sys
sys.path.insert(0, '/Users/jamienoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)

import astropy.convolution as astroconv
import numpy as np

iLength = 10000
jLength = iLength
image = np.random.random((iLength, jLength))

#image[50, 50] = np.nan

size = 11#1
ker = astroconv.Gaussian2DKernel(20,x_size=size,y_size=size)
#kerr = np.full((size,size), 10000)







<module 'astropy.version' from '/Users/jamienoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/astropy/version.py'>


In [3]:
%time res = astroconv.convolve(image, ker, boundary=None, normalize_kernel=False)
%time res_dev = astroconv.convolve_dev(image, ker, boundary=None, normalize_kernel=False)

CPU times: user 20.8 s, sys: 329 ms, total: 21.2 s
Wall time: 21.2 s
CPU times: user 21.2 s, sys: 338 ms, total: 21.6 s
Wall time: 21.6 s


In [ ]:
res = astroconv.convolve(image, ker, boundary=None)
res_dev = astroconv.convolve_dev(image, ker, boundary=None)

In [ ]:
import matplotlib.pyplot as plt
#mport matplotlib
%matplotlib qt5

plt.imshow(res, cmap='gray')
plt.colorbar()

In [ ]:
#delta = res == res_dev
#delta.all()
d = res_dev - res
np.isclose(d, 0, atol=1e-8).all()

In [ ]:

import matplotlib.pyplot as plt
#mport matplotlib
%matplotlib qt5

plt.imshow(d, cmap='gray')
plt.colorbar()